### Census Tract Issue - USDA Food Access Research Atlas (Tennessee data only)

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np


Importing both datasets

In [ ]:
tn_food_access_df = pd.read_csv('../data/tn_food_access.csv')

In [ ]:
tn_food_access_df = tn_food_access_df.drop('State', axis=1)

In [ ]:
# I need to see if this is doing something constructive with the data, or anything at all, before running:
# tn_food_access_df = tn_food_access_df.replace(r'^\s*$', np.nan, regex=True)

In [ ]:
# tn_food_access_df_fillna.to_csv('tn_food_access_nan.csv')

Getting a general sense of the way census tracts operate in the two datasets ...

In [ ]:
tn_food_access_df.head()

In [ ]:
tn_food_access_df['CT_Clean'].info()

993 unique entries for census tract, yet 1,489 total? Need to look into this as a top priority.

In [ ]:
tn_food_access_df['CT_Clean'].nunique()

In [ ]:
# https://saturncloud.io/blog/how-to-check-for-duplicate-values-in-pandas-dataframe-column/#:~:text=To%20check%20for%20duplicate%20values%20in%20a%20Pandas%20DataFrame%20column,is%20a%20duplicate%20or%20not.
# duplicate_values = df['Product'].duplicated()
# print(duplicate_values)

duplicate_CTs = tn_food_access_df['CT_Clean'].duplicated()
duplicate_CTs_to_csv = pd.DataFrame(duplicate_CTs)
# duplicate_CTs_to_csv.to_csv('duplicate_CTs.csv')


ISSUE: converting the census tract numbers to match the census reporter data did not work and created duplicate census tract IDs in about 1/3 of the data. I will probably need to go back and change some things but need some help troubleshooting and figuring out where to start/what's needed.

In [ ]:
tn_food_access_df['CT_Old'].info()

In [ ]:
tn_food_access_df['CT_Old'].nunique()

#### Origin of issue discovery:

Pulling minimum population, I'm seeing two lines that appear to have the same census tract number. 

In [ ]:
tn_food_access_df[tn_food_access_df['Pop2010'].iloc[:]==tn_food_access_df['Pop2010'].iloc[:].min()]

After speaking with instructors:
* pull digits only from census_df
* pull out county, create column for that
* make a list out of all column headers
* expand/true on comma
* take CT_Old column from tn_food_access_df and compare
* be sure to add "from income" to census_df and "from food" to tn_food_access_df
* union and catch for duplicates
* see the stuff that only exists in one form or another
* maybe also research census tract numbers (do they come in other forms? why wouldn't they be the same? is one more granular?)

In [ ]:
census_df = pd.read_excel('../data/census_tn_income_b19001.xlsx')

In [ ]:
census_df.head()

In [ ]:
census_df_ct_troubleshoot = census_df.rename(columns={'Unnamed: 0':'income_bracket'})

In [ ]:
census_df_ct_troubleshoot = census_df_ct_troubleshoot[census_df_ct_troubleshoot.columns.drop(list(census_df_ct_troubleshoot.filter(regex='Unnamed')))]


In [ ]:
census_df_ct_troubleshoot.head()

In [ ]:
# this removes the first row of data
census_df_ct_troubleshoot = census_df_ct_troubleshoot.iloc[1:,:]

In [ ]:
census_df_ct_troubleshoot.head()

In [ ]:
census_df_ct_troubleshoot = census_df_ct_troubleshoot.transpose().reset_index()

In [ ]:
census_df_ct_troubleshoot.columns = census_df_ct_troubleshoot.iloc[0]

In [ ]:
census_df_ct_troubleshoot = census_df_ct_troubleshoot.iloc[1:,:]

In [ ]:
census_df_ct_troubleshoot['income_bracket'].replace('Census Tract ','', regex=True, inplace = True)

In [ ]:
# census_df_ct_troubleshoot_test = census_df_ct_troubleshoot['income_bracket'].replace('Census Tract ','', inplace=True)

I've successfully taken "Census Tract" out of the income_bracket column. Now I need to create a new column and use the .split() function to separate the income bracket column into two separate columns: one for the census tract number, and one for the county name. Then I probably want another column (instead of altering the original ct number) which removes the decimal points from the census tracts. 

In [ ]:
census_df_ct_troubleshoot.head()

In [ ]:
census_df_ct_troubleshoot.columns.values.tolist()

In [ ]:
# df[['First Name', 'Last Name']] = df['Name'].str.split(' ', expand=True)
# https://saturncloud.io/blog/how-to-split-one-column-into-multiple-columns-in-pandas-dataframe/

census_df_ct_troubleshoot[['income_bracket','county_cr']] = census_df_ct_troubleshoot['income_bracket'].str.split(',',n=1,expand=True)
census_df_ct_troubleshoot.head()



In [ ]:
census_df_ct_troubleshoot['county_cr'].replace(', TN','', regex=True, inplace = True)

In [ ]:
census_df_ct_troubleshoot.head()

In [ ]:
census_df_ct_troubleshoot = census_df_ct_troubleshoot.rename(columns={'income_bracket':'census_tract','Total:':'total'})

In [ ]:
census_df_ct_troubleshoot.head()

In [ ]:
census_df_ct_troubleshoot['ct_digits_cr'] = census_df_ct_troubleshoot['census_tract'].str.replace('.','')

In [ ]:
census_df_ct_troubleshoot.head(200)

In [ ]:
census_df_ct_troubleshoot['from'] = 'census'

In [ ]:
census_df_ct_troubleshoot.head()

In [ ]:
tn_food_access_df.head()

In [ ]:
# census_df_ct_troubleshoot = census_df_ct_troubleshoot.rename(columns={'income_bracket':'census_tract','Total:':'total'})

tn_food_access_df = tn_food_access_df.rename(columns={'CT_Old':'ct_long_usda','CT_Clean':'ct_error_usda','County':'county_usda'})

In [ ]:
tn_food_access_df.head()

In [ ]:
# census_df_ct_troubleshoot['from'] = 'census'
tn_food_access_df['from'] = 'usda'

In [ ]:
tn_food_access_df['county_usda'].replace(' County','', regex=True, inplace = True)

In [ ]:
usda_data_to_union = tn_food_access_df[['ct_long_usda','county_usda','from']].copy()

In [ ]:
tn_food_access_df.head()

In [ ]:
usda_data_to_union.head()

In [ ]:
cr_data_to_union = census_df_ct_troubleshoot[['ct_digits_cr','county_cr','from']].copy()

In [ ]:
# this removes the first row of data
cr_data_to_union = cr_data_to_union.iloc[1:,:]
cr_data_to_union.head()

In [ ]:
# https://saturncloud.io/blog/how-to-perform-a-union-of-two-pandas-dataframes/
joined_usda_cr_tracts = pd.merge(usda_data_to_union,cr_data_to_union, how='outer',on=['ct_long_usda','ct_digits_cr'])

In [ ]:
joined_usda_cr_tracts.head(50)